In [1]:
import json
from urllib.parse import urlencode
from urllib.request import urlretrieve
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import requests
from lxml import html
import re
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

idx = pd.IndexSlice

In [2]:
def get_nba_data(endpt, params, return_url=False):

    ## endpt: https://github.com/seemethere/nba_py/wiki/stats.nba.com-Endpoint-Documentation
    ## params: dictionary of parameters: i.e., {'LeagueID':'00'}

    useragent = "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9\""
    dataurl = "\"" + "http://stats.nba.com/stats/" + endpt + "?" + urlencode(params) + "\""
    
    # for debugging: just return the url
    if return_url:
        return(dataurl)
    
    jsonstr = !wget -q -O - --user-agent={useragent} {dataurl}
    
    data = json.loads(jsonstr[0])
    
    h = data['resultSets'][0]['headers']
    d = data['resultSets'][0]['rowSet']
    
    return(pd.DataFrame(d, columns=h))

def get_params(url):
    endpt = url.split('stats/')[1].split('?')[0]
    params = {}
    paramstring = url.split('stats/')[1].split('?')[1]

    for substring in paramstring.split("&"):
        variable = substring.split('=')[0]
        value = substring.split('=')[1]
        params.update({variable:value})
    return([endpt,params])
user_agent = "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9\""

headers = {'User-Agent':user_agent}

def SportsRefCBBRowMaker(player_url,table_list,player_name,correct_year,player_id="0"):
    
    page = requests.get(player_url,headers=headers)
    tree = html.fromstring(page.content)
    
    if '404' in tree.xpath('//title')[0].text:
        return('404')
    else:
        year = tree.xpath('//table/tbody/tr')[-1].items()[0][-1][-4:]
        if(year!=correct_year):
            return(year)
        
        for i,table_name in enumerate(table_list):
            path = '//div[@id="'+table_name+'"]'
            try:
                treeStr = str(tree.xpath(path)[0].getchildren()[-1])[8:-4]
            except:
                continue
            #This is tricky and comments might be nice here. 

            tree2 = html.fromstring(treeStr)

            valuesHTML = tree2.xpath('//table/tfoot/tr')[-1].getchildren()[1:]
            ## This gives career in particular 
            conf = tree2.xpath('//table/tbody/tr/td')[1].getchildren()[0].text
            colNamesHTML = tree2.xpath('//table/thead/tr/th')
            colNames = ["PLAYER_ID","PLAYER_NAME"]+["CONF"] + [col.text for col in colNamesHTML][1:]
            values = [player_id,player_name] + [conf] + [val.text for val in valuesHTML]
            if(i==0):
                row = pd.DataFrame(pd.Series(values,index=colNames)\
                    .dropna()).transpose()
            else:
                row = pd.merge(row,pd.DataFrame(pd.Series(values,index=colNames)\
                    .dropna()).transpose())
        return(row)
    
def sportsRefCBBTableMaker(player_df, table_list):
    base_url = 'https://www.sports-reference.com/cbb/players/'
    columns_df_url = "https://www.sports-reference.com/cbb/players/malcolm-brogdon-1.html"
    columns_df = SportsRefCBBRowMaker(columns_df_url,table_list
                                      ,"Malcolm Brogdon(2016)",'2016')
    CBB_df = pd.DataFrame(columns=columns_df.columns)
    for index,row in player_df.iterrows():
        player_name = row['PLAYER_NAME']
        first = re.sub('[^a-zA-Z]+', '',player_name.split()[0].lower())
        last = "".join(player_name.split()[1:])[0:-6].lower().strip(".")
        year = player_name.split()[1][-5:-1]
        name_url = base_url + first + "-" + last + "-"

        i = 1
        stopping_condition=False
        while(stopping_condition==False):
            data_url = name_url + str(i) + ".html"
#            print(data_url)
            row = SportsRefCBBRowMaker(data_url,table_list,player_name,year,player_id=index)
            if isinstance(row, pd.DataFrame):
                stopping_condition=True
                continue
            if(row=='404'):
                stopping_condition=True
                row = None
                if(last[-2:]=="jr"):
                    last = last[:-2]
                    name_url = base_url + first + "-" + last + "-"
                    i=1
                    stopping_condition = False 
            i+=1
        CBB_df = pd.concat([CBB_df,row])
    return(CBB_df)

In [3]:
main_df = pickle.load(open('NBA_2014_18_Adv_100_Score_Def.pkl','rb'))
Player_ID_Dict = pickle.load(open('NBAPlayerDict.pkl','rb'))

In [4]:
main_df.head(15)

PLAYER_NAME   AGE  AST_PCT  AST_RATIO  \
PLAYER_ID SEASON SEASON_TYPE                                                 
101106    2014   Regular Season     Andrew Bogut  29.0    0.087       18.3   
          2015   Playoffs           Andrew Bogut  30.0    0.121       25.2   
                 Regular Season     Andrew Bogut  30.0    0.149       27.3   
          2016   Playoffs           Andrew Bogut  31.0    0.127       22.7   
                 Regular Season     Andrew Bogut  31.0    0.145       29.7   
          2017   Regular Season     Andrew Bogut  32.0    0.133       27.6   
          2018   Regular Season     Andrew Bogut  33.0    0.099       25.1   
101107    2014   Regular Season  Marvin Williams  28.0    0.075       11.3   
          2015   Regular Season  Marvin Williams  29.0    0.078       14.4   
          2016   Playoffs        Marvin Williams  30.0    0.043        9.9   
                 Regular Season  Marvin Williams  30.0    0.074       11.2   
          2017   Regular Season  Marvin Williams  31.0    0.072       11.3   
          2018   Regular Season  Marvin Williams  31.0    0.069       12.3   
101108    2014   Playoffs             Chris Paul  29.0    0.427       34.1   
                 Regular Season       Chris Paul  29.0    0.459       36.2   

                                 AST_TO  DEF_RATING  DREB_PCT  EFG_PCT  GP  \
PLAYER_ID SEASON SEASON_TYPE                                                 
101106    2014   Regular Season    1.15        98.8     0.293    0.627  67   
          2015   Playoffs          1.38        98.9     0.275    0.560  19   
                 Regular Season    1.70        95.2     0.262    0.563  67   
          2016   Playoffs          1.58        95.4     0.203    0.623  22   
                 Regular Season    1.95        97.2     0.252    0.629  70   
          2017   Regular Season    1.14        97.4     0.320    0.469  27   
          2018   Regular Season    0.79        99.7     0.251    0.680  23   
101107    2014   Regular Season    1.47       110.4     0.179    0.519  66   
          2015   Regular Season    1.67       102.2     0.183    0.520  78   
          2016   Playoffs          3.00       109.5     0.225    0.333   7   
                 Regular Season    1.77       102.6     0.187    0.554  81   
          2017   Regular Season    1.77       106.4     0.202    0.510  76   
          2018   Regular Season    1.45       105.5     0.155    0.570  78   
101108    2014   Playoffs          3.44       107.6     0.114    0.548  13   
                 Regular Season    4.57       100.1     0.114    0.511  62   

                                  MIN     ...      DEF_FREQ_3Pt  \
PLAYER_ID SEASON SEASON_TYPE              ...                     
101106    2014   Regular Season  26.4     ...             0.064   
          2015   Playoffs        23.1     ...             0.055   
                 Regular Season  23.6     ...             0.065   
          2016   Playoffs        16.6     ...             0.131   
                 Regular Season  20.7     ...             0.082   
          2017   Regular Season  21.6     ...             0.084   
          2018   Regular Season   9.4     ...             0.177   
101107    2014   Regular Season  25.4     ...             0.263   
          2015   Regular Season  26.1     ...             0.245   
          2016   Playoffs        32.5     ...             0.367   
                 Regular Season  28.9     ...             0.293   
          2017   Regular Season  30.2     ...             0.362   
          2018   Regular Season  25.7     ...             0.331   
101108    2014   Playoffs        36.3     ...             0.383   
                 Regular Season  35.0     ...             0.352   

                                 DEF_FREQ_Gt15  DEF_FREQ_Lt10  DEF_FREQ_Lt6  \
PLAYER_ID SEASON SEASON_TYPE                                                  
101106    2014   Regular Season          0.267          0.628         0.483   
      

In [7]:
draft_history = get_nba_data('drafthistory',{'LeagueID':'00'})

In [8]:
draft_history['SEASON']=pd.Series(draft_history.SEASON,dtype='int64')
draft_history = draft_history.query('SEASON>=2010')
draft_history = draft_history.query("ORGANIZATION_TYPE == 'College/University'")

draft_history['PLAYER_ID']=pd.Series(draft_history['PERSON_ID'],dtype=str)

draft_history = draft_history.set_index('PLAYER_ID')
#draft_history = draft_history.drop('PERSON_ID',axis=1)

draft_history.head()

,PERSON_ID,PLAYER_NAME,SEASON,ROUND_NUMBER,ROUND_PICK,OVERALL_PICK,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,ORGANIZATION,ORGANIZATION_TYPE
PLAYER_ID,,,,,,,,,,,,
1628365,1628365,Markelle Fultz,2017,1,1,1,1610612755,Philadelphia,76ers,PHI,Washington,College/University
1628366,1628366,Lonzo Ball,2017,1,2,2,1610612747,Los Angeles,Lakers,LAL,California-Los Angeles,College/University
1628369,1628369,Jayson Tatum,2017,1,3,3,1610612738,Boston,Celtics,BOS,Duke,College/University
1628367,1628367,Josh Jackson,2017,1,4,4,1610612756,Phoenix,Suns,PHX,Kansas,College/University
1628368,1628368,De'Aaron Fox,2017,1,5,5,1610612758,Sacramento,Kings,SAC,Kentucky,College/University


In [10]:
qualified_player_list = main_df.loc[idx[:,:,'Regular Season'],:].reset_index().query('SEASON_MIN>700').PLAYER_ID.unique()

In [11]:
combine_columns = ['PLAYER_NAME','DRAFT_YEAR','POSITION','STANDING_VERTICAL_LEAP',
                   'MAX_VERTICAL_LEAP','LANE_AGILITY_TIME',
                   'MODIFIED_LANE_AGILITY_TIME','THREE_QUARTER_SPRINT',
                   'BENCH_PRESS','HEIGHT_WO_SHOES','WEIGHT','WINGSPAN',
                   'STANDING_REACH','BODY_FAT_PCT','HAND_LENGTH','HAND_WIDTH'                  
                  ]
params = {'LeagueID':'00'}
full_combine_df = pd.DataFrame(columns = combine_columns)
for i in range(8,19):
    year="20"+format(i,'02d')+"-"+format(i+1,'02d')
    params.update({"SeasonYear":year})
    df1 = get_nba_data(endpt="draftcombinedrillresults",params=params)
    df2 = get_nba_data(endpt="draftcombineplayeranthro",params=params)
    dfMerged = pd.merge(df1,df2)
    dfMerged['PLAYER_ID']=pd.Series(dfMerged.PLAYER_ID,dtype=str)
    if(i>13):
        dfMerged['PLAYER_NAME'] = dfMerged['PLAYER_NAME']
        dfMerged['PLAYER_ID']=(dfMerged['PLAYER_NAME']+"("+year[0:4]+")").map(Player_ID_Dict)
        dfMerged['PLAYER_ID']=pd.Series(dfMerged['PLAYER_ID'],dtype=str).str.split(".").str.get(0)

    dfMerged['DRAFT_YEAR'] = int(year[0:4])    
    dfMerged = dfMerged[dfMerged['PLAYER_ID']!='nan']

    dfMerged = dfMerged.set_index('PLAYER_ID')[combine_columns]
        
    full_combine_df = pd.concat([full_combine_df,dfMerged])
full_combine_df.fillna(value=np.nan, inplace=True)
full_combine_df = full_combine_df.drop(['MODIFIED_LANE_AGILITY_TIME','BENCH_PRESS'],axis=1)

In [12]:
full_combine_df.tail()

,PLAYER_NAME,DRAFT_YEAR,POSITION,STANDING_VERTICAL_LEAP,MAX_VERTICAL_LEAP,LANE_AGILITY_TIME,THREE_QUARTER_SPRINT,HEIGHT_WO_SHOES,WEIGHT,WINGSPAN,STANDING_REACH,BODY_FAT_PCT,HAND_LENGTH,HAND_WIDTH
1628403,Caleb Swanigan,2017,PF,NaN,NaN,NaN,NaN,79.50,245.6,87.0,108.0,NaN,9.50,10.25
1628414,Sindarius Thornwell,2017,SG-SF,27.0,30.5,11.48,3.36,75.50,211.6,82.0,103.0,7.8,8.75,8.75
1628476,Derrick Walton Jr.,2017,PG,26.0,32.5,11.28,3.29,71.00,188.6,74.5,95.0,5.8,8.00,8.50
1628401,Derrick White,2017,PG,31.0,36.5,10.84,3.08,75.25,189.8,79.5,101.5,6.2,8.25,8.50
1628391,D.J. Wilson,2017,PF,NaN,NaN,NaN,NaN,80.75,234.4,87.0,109.5,6.4,9.25,10.25


In [13]:
qualified_player_list = main_df.loc[idx[:,:,'Regular Season'],:].reset_index().query('SEASON_MIN>700').PLAYER_ID.unique()
new_player_list=np.intersect1d(qualified_player_list,np.array(full_combine_df.index))

In [14]:
qual_combine_df  = full_combine_df.loc[new_player_list,:]
len(qual_combine_df)

242

We have 200 players drafted after 2010 who have full combien results and  played enough minutes in at least one season to qualify. 
Not sure if this is going to be enough but lets work off it. Now 189 for players who were drafted.

Lets first pull based on college to minimize excessive bball ref requests.

Saves 50 out of 200 requests?
Lets just move on 

In [ ]:
rowAdvanced = SportsRefCBBRowMaker('https://www.sports-reference.com/cbb/players/malcolm-brogdon-1.html',
                         ['all_players_per_poss','all_players_advanced'],'Malcolm Brogdon(2016)','2016')

In [ ]:
qual_combine_df['PLAYER_NAME'] = qual_combine_df['PLAYER_NAME'] + "(" +\
            pd.Series(qual_combine_df.DRAFT_YEAR,dtype=str).str[:4] + ")"
qual_combine_df.PLAYER_NAME

In [ ]:
college_stats_df=sportsRefCBBTableMaker(qual_combine_df,['all_players_per_min','all_players_advanced'])

No per 100 possesion from 2010 is interseting. Lets go back to draft and see what to do, Ok lets just do per 40 minutes instead

In [ ]:
#college_stats_df = college_stats_df.set_index('PLAYER_ID')

In [ ]:
college_draft_prospects = pd.merge(college_stats_df,full_combine_df,left_index=True,right_index=True)

In [ ]:
df = college_draft_prospects.apply(pd.to_numeric,errors='ignore')
df = full_combine_df
df.POSITION.unique()

Lets reposition to Guard, Wing, Forward, Big

Lets fill in missing data here. Lets do it by average of position if we can figure that out.

In [ ]:
df['POSITION_SIMPLE']=df.POSITION
df.POSITION_SIMPLE[df.POSITION_SIMPLE.str.contains('C')]='Big'
df.POSITION_SIMPLE[df.POSITION_SIMPLE.str.contains('PF')]='Forward'
df.POSITION_SIMPLE[df.POSITION_SIMPLE.str.contains('SF')]='Wing'
df.POSITION_SIMPLE[df.POSITION_SIMPLE.str.contains('G')]='Guard'
df.loc[:,['POSITION','POSITION_SIMPLE']]

Start working on handling missing values.

Fix combine data based on new position averages.
Instead maybe we do a linear model of weight, and height?

In [ ]:
df.count()
df.loc[:,['STANDING_VERTICAL_LEAP','MAX_VERTICAL_LEAP','LANE_AGILITY_TIME',\
   'THREE_QUARTER_SPRINT','HAND_LENGTH','HAND_WIDTH','WEIGHT','HEIGHT_WO_SHOES','POSITION_SIMPLE']]\
    .groupby('POSITION_SIMPLE').count()


In [ ]:
df.columns

Lets take a little step back and focus on filling out data a little more for those who missed combine.

We'll pull height weight and position then impute missing values that way

In [17]:
url = 'http://stats.nba.com/stats/leaguedashplayerbiostats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=F&Season=2017-18&SeasonSegment=&SeasonType=Playoffs&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='
endpt,params = get_params(url)
get_nba_data(endpt,params).head()

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,PLAYER_HEIGHT,PLAYER_HEIGHT_INCHES,PLAYER_WEIGHT,COLLEGE,COUNTRY,...,GP,PTS,REB,AST,NET_RATING,OREB_PCT,DREB_PCT,USG_PCT,TS_PCT,AST_PCT
0,201166,Aaron Brooks,1610612750,MIN,33.0,6-0,72,161,Oregon,USA,...,2,2.0,0.0,0.0,0.0,0.000,0.000,0.564,0.515,0.000
1,1628935,Aaron Jackson,1610612745,HOU,32.0,6-3,75,183,None,USA,...,7,0.7,0.6,0.3,-22.7,0.111,0.118,0.183,0.512,0.200
2,1627846,Abdel Nader,1610612738,BOS,24.0,6-6,78,230,Iowa State,Egypt,...,11,1.1,0.3,0.3,-24.5,0.030,0.065,0.257,0.358,0.188
3,201143,Al Horford,1610612738,BOS,32.0,6-10,82,245,Florida,Dominican Republic,...,19,15.7,8.3,3.3,4.2,0.054,0.216,0.185,0.631,0.155
4,202329,Al-Farouq Aminu,1610612757,POR,27.0,6-9,81,220,Wake Forest,USA,...,4,17.3,9.0,1.3,-16.3,0.087,0.220,0.188,0.652,0.062


Position is going to be slightly more complicated but we have a plan!

In [19]:
drafted_from_college = np.array(draft_history.index)
from_college_qual = np.intersect1d(qualified_player_list,drafted_from_college)

In [20]:
combine_player_list = np.array(full_combine_df.index)

players_to_find = pd.Series(np.setdiff1d(from_college_qual,combine_player_list))

In [21]:
players_to_find_df = draft_history.loc[players_to_find,:][['PLAYER_NAME','SEASON','TEAM_ID','TEAM_NAME']]

In [22]:
endpt = 'commonteamroster'
all_players = get_nba_data('commonallplayers',{'LeagueID':'00','Season':'2017-18','IsOnlyCurrentSeason':'0'})
player_team_dict = all_players.set_index('PERSON_ID')['TEAM_ID']
new_player_combine_df = pd.DataFrame(columns=full_combine_df.columns)

for index,row in players_to_find_df.iterrows():
    season = str(row.SEASON)[:4]+"-"+str(row.SEASON+1)[2:]
    try:
        params = {'Season':season,'TeamID':str(row.TEAM_ID)}
        df_temp = get_nba_data(endpt,params)
        df_temp = df_temp.set_index('PLAYER_ID')
        height = df_temp.HEIGHT[int(index)].split("-")
        height_inches = int(height[0])*12 + int(height[1])-1
        weight = df_temp.WEIGHT[int(index)]
        position = df_temp.POSITION[int(index)]
#        print('trying')
    except:
#        print('except')
        try:
            team_id = player_team_dict[int(index)]
        except:
            continue
        params = {'Season':'2017-18','TeamID':str(team_id)}
        season_curr = 2017
        while(team_id==0 and season_curr>=2010):
            season_str = str(season_curr)[:4]+"-"+str(season_curr+1)[2:]
            all_players_temp = get_nba_data('commonallplayers',{'LeagueID':'00','Season':season_str,
                                                           'IsOnlyCurrentSeason':'1'})
            player_team_dict_temp = all_players_temp.set_index('PERSON_ID')['TEAM_ID']
            try:
                team_id = player_team_dict_temp[int(index)]
            except: 
                break
            params = {'Season':season_str,'TeamID':str(team_id)}
            season_curr-=1
#            print(season_curr)
#            print(season_str)
        try:
            df_temp = get_nba_data(endpt,params)
        except:
            continue
        df_temp = df_temp.set_index('PLAYER_ID')
        height = df_temp.HEIGHT[int(index)].split("-")
        height_inches = int(height[0])*12 + int(height[1])-1
        weight = df_temp.WEIGHT[int(index)]
        position = df_temp.POSITION[int(index)]
    new_row = pd.Series({'PLAYER_ID':index,
                         'PLAYER_NAME':row.PLAYER_NAME,
                         'DRAFT_YEAR':row.SEASON,
                         'HEIGHT_WO_SHOES':height_inches,
                         'WEIGHT':weight,'POSITION':position
                        }).transpose()
    new_player_combine_df=new_player_combine_df.append(new_row,ignore_index=True)
#    print([height,weight,position])
new_player_combine_df = new_player_combine_df.set_index('PLAYER_ID')

In [23]:
new_player_combine_df

,PLAYER_NAME,DRAFT_YEAR,POSITION,STANDING_VERTICAL_LEAP,MAX_VERTICAL_LEAP,LANE_AGILITY_TIME,THREE_QUARTER_SPRINT,HEIGHT_WO_SHOES,WEIGHT,WINGSPAN,STANDING_REACH,BODY_FAT_PCT,HAND_LENGTH,HAND_WIDTH
PLAYER_ID,,,,,,,,,,,,,,
1626143,Jahlil Okafor,2015,C,NaN,NaN,NaN,NaN,82,275,NaN,NaN,NaN,NaN,NaN
1626150,Andrew Harrison,2015,G,NaN,NaN,NaN,NaN,77,213,NaN,NaN,NaN,NaN,NaN
1626156,D'Angelo Russell,2015,G,NaN,NaN,NaN,NaN,76,195,NaN,NaN,NaN,NaN,NaN
1626157,Karl-Anthony Towns,2015,C-F,NaN,NaN,NaN,NaN,83,244,NaN,NaN,NaN,NaN,NaN
1626162,Kelly Oubre Jr.,2015,F,NaN,NaN,NaN,NaN,78,205,NaN,NaN,NaN,NaN,NaN
1626196,Josh Richardson,2015,G,NaN,NaN,NaN,NaN,77,200,NaN,NaN,NaN,NaN,NaN
1626204,Larry Nance Jr.,2015,F,NaN,NaN,NaN,NaN,80,230,NaN,NaN,NaN,NaN,NaN
1627732,Ben Simmons,2016,F,NaN,NaN,NaN,NaN,81,240,NaN,NaN,NaN,NaN,NaN
1627734,Domantas Sabonis,2016,F,NaN,NaN,NaN,NaN,82,240,NaN,NaN,NaN,NaN,NaN


In [24]:
full_combine_df = pd.concat([full_combine_df,new_player_combine_df])

In [31]:
full_combine_df = pickle.load(open('CurrentCombineData.pkl','rb'))
full_combine_df.head()

,PLAYER_NAME,DRAFT_YEAR,POSITION,STANDING_VERTICAL_LEAP,MAX_VERTICAL_LEAP,LANE_AGILITY_TIME,THREE_QUARTER_SPRINT,HEIGHT_WO_SHOES,WEIGHT,WINGSPAN,STANDING_REACH,BODY_FAT_PCT,HAND_LENGTH,HAND_WIDTH,POSITION_SIMPLE
201570,Joe Alexander,2008,SF-PF,32.5,38.5,11.33,2.99,79.25,220.0,83.50,106.0,5.8,NaN,NaN,Forwad
201589,Darrell Arthur,2008,PF,28.5,30.0,12.18,3.14,79.50,216.0,82.75,107.0,7.3,NaN,NaN,Forwad
201571,D.J. Augustin,2008,PG,28.5,35.0,11.27,3.07,70.00,171.5,75.50,94.5,8.1,NaN,NaN,Guard
201573,Jerryd Bayless,2008,PG-SG,31.0,38.0,11.26,3.07,73.75,204.0,75.50,97.0,4.7,NaN,NaN,Guard
201563,Michael Beasley,2008,PF-SF,30.0,35.0,11.06,3.24,79.00,239.0,84.25,107.0,7.7,NaN,NaN,Forwad


In [32]:
full_combine_df['POSITION_SIMPLE']=full_combine_df.POSITION
full_combine_df.POSITION_SIMPLE[full_combine_df.POSITION_SIMPLE.str.contains('C')]='Big'
full_combine_df.POSITION_SIMPLE[full_combine_df.POSITION_SIMPLE.str.contains('PF')]='Forward'
full_combine_df.POSITION_SIMPLE[full_combine_df.POSITION_SIMPLE.str.contains('SF')]='Wing'
full_combine_df.POSITION_SIMPLE[full_combine_df.POSITION_SIMPLE.str.contains('G')]='Guard'
full_combine_df.POSITION_SIMPLE[full_combine_df.POSITION_SIMPLE.str.contains('F')]='Forwad'

full_combine_df.loc[:,['POSITION','POSITION_SIMPLE']]

/home/dhoskins/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/dhoskins/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/dhoskins/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/dh

,POSITION,POSITION_SIMPLE
201570,SF-PF,Forwad
201589,PF,Forwad
201571,PG,Guard
201573,PG-SG,Guard
201563,PF-SF,Forwad
12239,PG-SG,Guard
12240,PG,Guard
12241,PF,Forwad
201812,SF-PF,Forwad
201868,PG,Guard


In [33]:
full_combine_df = full_combine_df.apply(pd.to_numeric,errors='ignore')
full_combine_df = full_combine_df.loc[~full_combine_df.WEIGHT.isnull(),:]
full_combine_df

,PLAYER_NAME,DRAFT_YEAR,POSITION,STANDING_VERTICAL_LEAP,MAX_VERTICAL_LEAP,LANE_AGILITY_TIME,THREE_QUARTER_SPRINT,HEIGHT_WO_SHOES,WEIGHT,WINGSPAN,STANDING_REACH,BODY_FAT_PCT,HAND_LENGTH,HAND_WIDTH,POSITION_SIMPLE
201570,Joe Alexander,2008,SF-PF,32.5,38.5,11.33,2.99,79.25,220.0,83.50,106.0,5.8,NaN,NaN,Forwad
201589,Darrell Arthur,2008,PF,28.5,30.0,12.18,3.14,79.50,216.0,82.75,107.0,7.3,NaN,NaN,Forwad
201571,D.J. Augustin,2008,PG,28.5,35.0,11.27,3.07,70.00,171.5,75.50,94.5,8.1,NaN,NaN,Guard
201573,Jerryd Bayless,2008,PG-SG,31.0,38.0,11.26,3.07,73.75,204.0,75.50,97.0,4.7,NaN,NaN,Guard
201563,Michael Beasley,2008,PF-SF,30.0,35.0,11.06,3.24,79.00,239.0,84.25,107.0,7.7,NaN,NaN,Forwad
12239,Ramel Bradley,2008,PG-SG,24.5,29.5,10.68,3.24,73.25,184.0,77.00,98.0,5.7,NaN,NaN,Guard
12240,Tyrone Brazelton,2008,PG,30.0,34.0,11.59,3.25,70.00,166.0,73.75,94.0,8.1,NaN,NaN,Guard
12241,Takais Brown,2008,PF,29.0,31.0,12.47,3.25,79.25,252.0,82.00,106.0,9.2,NaN,NaN,Forwad
201812,Keith Brumbaugh,2008,SF-PF,26.0,27.0,11.65,3.28,79.50,190.0,79.25,104.5,4.0,NaN,NaN,Forwad
201868,Stanley Burrell,2008,PG,29.5,34.5,11.19,3.19,73.50,201.0,78.25,97.5,5.6,NaN,NaN,Guard


In [34]:
full_combine_df.groupby('POSITION_SIMPLE').describe().stack()

BODY_FAT_PCT   DRAFT_YEAR  HAND_LENGTH  HAND_WIDTH  \
POSITION_SIMPLE                                                             
Big             count     76.000000    82.000000    56.000000   56.000000   
                mean       8.519737  2011.853659     9.138393    9.879464   
                std        3.293661     2.931826     0.398633    0.793426   
                min        4.150000  2008.000000     8.250000    7.750000   
                25%        6.075000  2010.000000     9.000000    9.437500   
                50%        7.750000  2012.000000     9.250000   10.000000   
                75%        9.600000  2013.750000     9.500000   10.500000   
                max       20.800000  2017.000000    10.250000   12.000000   
Forwad          count    131.000000   148.000000    97.000000   97.000000   
                mean       8.187405  2012.324324     8.976804    9.677835   
                std        2.667766     3.042144     0.380314    0.737711   
                min        3.600000  2008.000000     8.000000    7.750000   
                25%        5.950000  2010.000000     8.750000    9.250000   
                50%        7.900000  2012.000000     9.000000    9.750000   
                75%        9.900000  2015.000000     9.250000   10.000000   
                max       15.500000  2017.000000    10.000000   11.750000   
Guard           count    195.000000   212.000000   146.000000  146.000000   
                mean       6.368205  2012.094340     8.446918    9.178082   
                std        1.811885     2.831871     0.357444    0.602408   
                min        3.000000  2008.000000     7.500000    7.500000   
                25%        5.200000  2010.000000     8.250000    8.750000   
                50%        6.050000  2012.000000     8.500000    9.250000   
                75%        7.300000  2014.000000     8.750000    9.500000   
                max       12.500000  2017.000000     9.250000   10.750000   
Wing            count     94.000000    94.000000    77.000000   77.000000   
                mean       6.984043  2012.489362     8.753247    9.399351   
                std        2.050390     2.857718     0.381508    0.612721   
                min        3.200000  2008.000000     8.000000    7.250000   
                25%        5.300000  2010.000000     8.500000    9.000000   
                50%        7.000000  2012.000000     8.750000    9.500000   
                75%        8.125000  2015.000000     9.000000    9.750000   
                max       13.350000  2017.000000     9.750000   11.250000   

                       HEIGHT_WO_SHOES  LANE_AGILITY_TIME  MAX_VERTICAL_LEAP  \
POSITION_SIMPLE                                                                
Big             count        82.000000          61.000000          61.000000   
                mean         81.881098          11.900492          31.483607   
                std           1.244882           0.589037           3.341615   
                min          78.250000          10.640000          25.000000   
                25%          81.000000          11.480000          29.000000   
                50%          81.875000          11.850000          32.000000   
                75%          82.750000          12.320000          33.500000   
                max          85.250000          13.440000          40.500000   
Forwad          count       148.000000         111.000000         114.000000   
                mean         79.792230          11.535856          34.122807   
                std           1.196078           0.513325           2.931792   
                min          75.750000          10.190000          27.000000   
                25%          79.000000          11.210000          32.000000   
                50%          79.750000          11.510000          34.500000   
                75%          80.562500          11.835000          36.000000   
                max          83.750000      

Now we begin to work on imputing. We're going to fit a predictive model for each feature based on height weight and positiion. Lets try lane agility time for starters.

Time to work on something that does this for the whole dataframe

In [35]:
predicting_vals = pd.get_dummies(full_combine_df[['DRAFT_YEAR','POSITION_SIMPLE','WEIGHT','HEIGHT_WO_SHOES']])
predicting_vals
cols_to_impute = ['STANDING_VERTICAL_LEAP','MAX_VERTICAL_LEAP','LANE_AGILITY_TIME',
                  'THREE_QUARTER_SPRINT','WINGSPAN','STANDING_REACH',
                  'BODY_FAT_PCT','HAND_LENGTH','HAND_WIDTH']

for col in cols_to_impute:
    print(col)
    missing = full_combine_df[col].isnull()
    X = predicting_vals[~missing]
    Y = full_combine_df[col][~missing]
    X_fill = predicting_vals[missing]
    model = LinearRegression(fit_intercept=True)
    model.fit(X,Y)
    full_combine_df[col][missing]=model.predict(X_fill)


STANDING_VERTICAL_LEAP
MAX_VERTICAL_LEAP


/home/dhoskins/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


LANE_AGILITY_TIME
THREE_QUARTER_SPRINT
WINGSPAN
STANDING_REACH
BODY_FAT_PCT
HAND_LENGTH
HAND_WIDTH


In [59]:
pickle.dump(full_combine_df,open('CombineImputed.pkl','wb'))

Now we get back to that college data loop we had earlier

In [47]:
qualified_player_list = main_df.loc[idx[:,:,'Regular Season'],:].reset_index().query('SEASON_MIN>700').PLAYER_ID.unique()
new_player_list=np.intersect1d(qualified_player_list,np.array(full_combine_df.index))
qual_combine_df  = full_combine_df.loc[new_player_list,:]
qual_combine_df['PLAYER_NAME'] = qual_combine_df['PLAYER_NAME'] + "(" +\
            pd.Series(qual_combine_df.DRAFT_YEAR,dtype=str).str[:4] + ")"
qual_combine_df.PLAYER_NAME
len(qual_combine_df)

278

In [48]:
college_stats_df=sportsRefCBBTableMaker(qual_combine_df,['all_players_per_min','all_players_advanced'])

In [53]:
college_draft_prospects = pd.merge(college_stats_df.set_index('PLAYER_ID'),
                                   qual_combine_df,left_index=True,right_index=True)
college_draft_prospects= college_draft_prospects.apply(pd.to_numeric,errors='ignore')

In [58]:
college_draft_prospects.describe()

,2P,2P%,2PA,3P,3P%,3PA,3PAr,AST,AST%,BLK,...,MAX_VERTICAL_LEAP,LANE_AGILITY_TIME,THREE_QUARTER_SPRINT,HEIGHT_WO_SHOES,WEIGHT,WINGSPAN,STANDING_REACH,BODY_FAT_PCT,HAND_LENGTH,HAND_WIDTH
count,252.000000,252.000000,252.000000,252.000000,239.000000,252.000000,252.000000,252.000000,250.000000,252.000000,...,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000
mean,5.280952,0.525770,9.986508,1.305159,0.333134,3.579365,0.256698,2.968254,15.268000,1.260714,...,35.491312,11.276968,3.282525,77.702381,216.277778,82.448533,103.310083,7.081166,8.726496,9.436924
std,1.601554,0.052844,2.660313,1.003824,0.107428,2.549567,0.173768,1.753046,8.424778,1.220217,...,2.993402,0.470154,0.112881,3.251603,24.148174,3.782652,4.798396,2.133857,0.425111,0.619137
min,2.200000,0.374000,4.000000,0.000000,0.000000,0.000000,0.000000,0.300000,1.800000,0.000000,...,25.000000,10.070000,3.010000,68.750000,149.200000,70.750000,88.500000,3.000000,7.500000,7.250000
25%,4.100000,0.491000,8.100000,0.300000,0.307000,1.200000,0.094000,1.700000,9.300000,0.400000,...,33.500000,10.980000,3.210000,75.500000,198.350000,80.000000,100.000000,5.400000,8.500000,9.000000
50%,5.100000,0.517000,9.900000,1.300000,0.356000,3.700000,0.272500,2.600000,13.000000,0.900000,...,35.612912,11.190000,3.274270,77.875000,215.900000,82.448560,103.500000,6.675000,8.750000,9.500000
75%,6.325000,0.562000,11.800000,2.100000,0.383000,5.600000,0.401500,3.700000,19.600000,1.800000,...,37.159538,11.541316,3.345726,80.000000,233.100000,85.250000,107.000000,8.362500,9.000000,9.806881
max,10.400000,0.732000,18.500000,4.900000,1.000000,11.900000,0.607000,11.100000,43.200000,8.200000,...,44.000000,12.770000,3.810000,84.250000,291.800000,91.000000,114.500000,16.400000,10.000000,11.750000


In [62]:
pickle.dump(college_draft_prospects,open('FullProspectData.pkl','wb'))